# The Query Feature

In [2]:
import mysql.connector
import pandas as pd

# For Query Screen

mydatabase = mysql.connector.connect(
    host="localhost",
    user="root",
    password="sid34",
    database="siddata"
)

mycursor = mydatabase.cursor()

mycursor.execute("SELECT * FROM client;")

myresult = mycursor.fetchall()

# printing the result

# for x in myresult:
#     print(x)

df = pd.read_sql_query('SELECT * FROM client;', mydatabase)

print(df)

   client_id          client_name  branch_id
0        402                FedEx          3
1        403  John Daliy Law, LLC          3
2        404  Scranton Whitepages          2
3        405      Times Newspaper          3
4        406                FedEx          2


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23236\73559188.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('SELECT * FROM client', mydatabase)


# Upload To S3 AWS

In [1]:
import boto3

# Get your AWS credentials
ACCESS_KEY_ID = "AKYZH"
SECRET_ACCESS_KEY = "F7O"

# Create a Boto3 client
s3 = boto3.client("s3", aws_access_key_id=ACCESS_KEY_ID, aws_secret_access_key=SECRET_ACCESS_KEY)

# Upload a file to S3
file_path = r"./Storage/sqlBackup_2023-05-07.sql"
bucket_name = "sidmybucket1234"
object_key = "sqlBackup_2023-05-07.sql"

s3.upload_file(file_path, bucket_name, object_key)

# Check if the file is uploaded successfully
response = s3.head_object(Bucket=bucket_name, Key=object_key)

if response["ResponseMetadata"]["HTTPStatusCode"] == 200:
    print("File uploaded successfully")
else:
    print("File upload failed")

File uploaded successfully


# Upload To Google Drive

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file('./data-store-1789-c5507a0628a0.json')

drive_service = build('drive', 'v3', credentials=credentials)

file_path = './Storage/sqlBackup_2023-05-08.sql'
# file_metadata = {
#     'name': 'My File SQL'
# }

media = MediaFileUpload(file_path)
file = drive_service.files().create(media_body=media, fields='id').execute()
print('File ID:', file.get('id'))

# Encryt the files

In [8]:
from cryptography.fernet import Fernet
import os

# Generate a random key
key = Fernet.generate_key()

filepath = "key.txt"

# Checking wether key.txt exists
if(os.path.isfile(filepath)):
    i = 1
    filepath = f"key{i}.txt"
    while(os.path.isfile(filepath)):
        i = i + 1
        filepath = f"key{i}.txt"

# Save the key to a file
with open(filepath, "wb") as f:
    f.write(key)

# Load the key from a file
with open(filepath, "rb") as f:
    key = f.read()

# Create a Fernet object
fernet = Fernet(key)

# opening the original file to encrypt
with open('./Storage/sqlBackup_2023-05-19.sql', 'rb') as file:
    original = file.read()

# encrypting the file
encrypted = fernet.encrypt(original)

# opening the file in write mode and
# writing the encrypted data
with open('./Storage/sqlBackup_2023-05-19.sql', 'wb') as encrypted_file:
    encrypted_file.write(encrypted)


In [10]:
def caesar_encrypt(realText, step):
    outText = []
    cryptText = []
    uppercase = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    lowercase = 'abcdefghijklmnopqrstuvwxyz'
    for eachLetter in realText:
        if eachLetter in uppercase:
            index = uppercase.index(eachLetter)
            crypting = (index + step) % 26
            cryptText.append(crypting)
            newLetter = uppercase[crypting]
            outText.append(newLetter)
        elif eachLetter in lowercase:
            index = lowercase.index(eachLetter)
            crypting = (index + step) % 26
            cryptText.append(crypting)
            newLetter = lowercase[crypting]
            outText.append(newLetter)
        elif eachLetter == " " or eachLetter == "\t" or eachLetter == "\n":
            outText.append(eachLetter)
    return outText

realFile = f"./Storage/sqlBackup_2023-05-19.sql"
step = int(input("Enter the Key: "))
with open(realFile, 'r') as f:
    text = f.read()
code = caesar_encrypt(text,step)
with open(realFile + ".encrypted", 'w') as f:
    for i in code:
        f.write(i)
print("The Encrypted Message is saved in", realFile + ".encrypted")

The Encrypted Message is saved in ./Storage/sqlBackup_2023-05-19.sql.encrypted


# Decrypt the files

In [9]:
# Select key file first to decrypt

with open("key2.txt", "rb") as f:
    key = f.read()

# Create a Fernet object
fernet = Fernet(key)

# opening the encrypted file
with open('./Storage/sqlBackup_2023-05-19.sql', 'rb') as enc_file:
    encrypted = enc_file.read()

# decrypting the file
decrypted = fernet.decrypt(encrypted)

# opening the file in write mode and
# writing the decrypted data
with open('./Storage/sqlBackup_2023-05-19.sql', 'wb') as dec_file:
    dec_file.write(decrypted)

In [11]:
def caesar_decrypt(realText, step):
    outText = []
    cryptText = []
    uppercase = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    lowercase = 'abcdefghijklmnopqrstuvwxyz'
    for eachLetter in realText:
        if eachLetter in uppercase:
            index = uppercase.index(eachLetter)
            crypting = (index - step) % 26
            cryptText.append(crypting)
            newLetter = uppercase[crypting]
            outText.append(newLetter)
        elif eachLetter in lowercase:
            index = lowercase.index(eachLetter)
            crypting = (index - step) % 26
            cryptText.append(crypting)
            newLetter = lowercase[crypting]
            outText.append(newLetter)
        elif eachLetter == " " or eachLetter == "\t" or eachLetter == "\n":
            outText.append(eachLetter)
    return outText

realFile = f"./Storage/sqlBackup_2023-05-19.sql.encrypted"
step = int(input("Enter the Key: "))
with open(realFile, 'r') as f:
    text = f.read()
code = caesar_decrypt(text,step)
with open(realFile + ".decrypted", 'w') as f:
    for i in code:
        f.write(i)
print("The Decrypted Message is saved in", realFile + ".decrypted")

The Decrypted Message is saved in ./Storage/sqlBackup_2023-05-19.sql.encrypted.decrypted


# The Convert Code

In [1]:
import mysql.connector
from pymongo import MongoClient

# Connect to the SQL database
sql_connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='sid34',
    database='siddata'
)

# Connect to the MongoDB database
mongo_client = MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['test']
mongo_collection = mongo_db['newH']

# Retrieve data from the SQL table
sql_cursor = sql_connection.cursor()
sql_cursor.execute('SELECT * FROM client')
sql_data = sql_cursor.fetchall()

# Transform and insert data into MongoDB collection
for row in sql_data:
    doc = {
        'field1': row[0],
        'field2': row[1],
        # Add more fields as needed, mapping SQL columns to MongoDB fields
    }
    mongo_collection.insert_one(doc)

# Close connections
sql_cursor.close()
sql_connection.close()
mongo_client.close()


# Save mongo to json

In [3]:
import json
from pymongo import MongoClient
from bson import json_util

# Connect to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')

# Access the MongoDB database and collection
db = client['test']
collection = db['newH']

# Retrieve the documents from the collection
documents = collection.find()

# Convert the documents to a list of dictionaries
document_list = [doc for doc in documents]

# Serialize the documents to JSON using json_util
json_data = json.dumps(document_list, default=json_util.default, indent=4)

# Save the JSON data to a file
with open('output.json', 'w') as file:
    file.write(json_data)

# Close the MongoDB connection
client.close()


# Backup MongoDB database

In [4]:
import subprocess
import datetime as dt
import os
import pathlib


def backup_mongodb(database, collection, output_file):
    """Backups the specified MongoDB database and collection to the specified output file.

    Args:
    database: The name of the database to backup.
    collection: The name of the collection to backup.
    output_file: The path to the output file.
    """

    command = ["mongodump",  "--db", database, "--collection", collection, "--out", output_file]
    subprocess.run(command)

def main():
    """Backs up the 'test' database and 'test_collection' collection to a file called 'backup.gz'.
    """

    curr = pathlib.Path().resolve()

    mongoBackupName = f"mongoBackup_{dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

    # Path
    path = os.path.join(curr, mongoBackupName)

    os.mkdir(path)

    backup_mongodb("test", "newH", path)


if __name__ == "__main__":
    main()


# MongoRestore

In [13]:
import subprocess
import shlex
from pathlib import Path
import os


def restore_mongodb_dump(dump_directory, db_host, db_port, db_name):

    # dump_directory = dump_directory.replace("\\", "/")
    # Wrap the dump_directory in quotes
    # dump_directory_quoted = shlex.quote(dump_directory)

    command = fr"mongorestore --host {db_host} --port {db_port} --db {db_name} {dump_directory}"
    try:
        subprocess.run(command, shell=True, check=True)
        print("MongoDB restore completed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during MongoDB restore: {e}")

# Example usage
# dump_directory = '"D:\\BRC System\\BRC Python\\mongoBackup_2023-06-24_23-15-27\\mongoBackup_2023-06-24_23-43-07"'

path_raw_string = r'"D:\BRC System\BRC Python\mongoBackup_2023-06-24_23-15-27\mongoBackup_2023-06-24_23-43-07"'
# dump_directory = fr"./mongoBackup_2023-06-24_23-15-27/mongoBackup_2023-06-24_23-43-07/"
# dump_directory = fr"./mongo Backup_2023-06-24_23-15-27/mongoBackup_2023-06-24_23-43-07"

# dump_path = os.path.abspath(dump_directory)

# print(dump_path)

db_host = "localhost"
db_port = 27017
db_name = "test"

# print(os.path.abspath("./mongo Backup_2023-06-24_23-15-27/mongoBackup_2023-06-24_23-43-07"))

restore_mongodb_dump(path_raw_string, db_host, db_port, db_name)


MongoDB restore completed successfully!


# Mongo Query


In [18]:
from pymongo import MongoClient

# Establish a connection to MongoDB
client = MongoClient("mongodb://localhost:27017")

# Access the database
db = client["test"]

# Choose the collection to query
collection = db["newH"]

# Define the query
query = {"field2": 'FedEx'}

# Execute the query
result = collection.find(query)

print(result)

# Process the query result
for document in result:
    print(document)

{'_id': ObjectId('6485f8db98b9eaffb3223c28'), 'field1': 402, 'field2': 'FedEx'}
{'_id': ObjectId('6485f8db98b9eaffb3223c2c'), 'field1': 406, 'field2': 'FedEx'}


# A feature which counts min & max field value in mongo database

In [2]:
from pymongo import MongoClient

# Establish a connection to MongoDB
client = MongoClient("mongodb://localhost:27017")

# Access the database
db = client["test"]

# Choose the collection
collection = db["newH"]

# Define the aggregation pipeline
pipeline = [
    {"$project": {"numFields": {"$size": {"$objectToArray": "$$ROOT"}}}},
    {"$group": {"_id": None, "minFields": {"$min": "$numFields"}, "maxFields": {"$max": "$numFields"}}}
]

# Execute the aggregation pipeline
result = list(collection.aggregate(pipeline))

# Process the result
if result:
    min_fields = result[0]["minFields"]
    max_fields = result[0]["maxFields"]
    print(f"Minimum number of fields: {min_fields}")
    print(f"Maximum number of fields: {max_fields}")
else:
    print("No documents found.")

Minimum number of fields: 3
Maximum number of fields: 4
